# Physics Informed Neural Networks <br> F1 Car Front Wing Aerodymanics

## PINN

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import csv
from pinn import PINN
import utils

In [ ]:
in_filepath = "/Users/ggito/repos/pinns/data/"
points_filename = "front_wing_points_final.csv"
norms_filename = "front_wing_norms_final.csv"

wing_df = pd.read_csv(in_filepath + points_filename)
norm_df = pd.read_csv(in_filepath + norms_filename)

print(wing_df)
print(norm_df)

In [ ]:
x_max = 1
y_max = 1
z_max = 1
t_max = 1

Nx = 10
Ny = 10
Nz = 10
Nt = 10

dx = x_max / (Nx - 1)
dy = y_max / (Ny - 1)
dz = z_max / (Nz - 1)
dt = t_max / (Nt - 1)

x_test = np.linspace(0, x_max, Nx)
y_test = np.linspace(0, y_max, Ny)
z_test = np.linspace(0, z_max, Nz)
t_test = np.linspace(0, t_max, Nt)

x_grid, y_grid, z_gripd, t_grid = np.meshgrid(x_test, y_test, z_test, t_test)

In [ ]:
input_dim = 4
output_dim = 4
# hidden_units = [32, 32, 32]
# hidden_units = [64, 64, 64, 64]
# hidden_units = [128, 128, 128, 128]
# hidden_units = [256, 256, 256, 256]
# hidden_units = [512, 512]
# hidden_units = [1024, 1024, 1024]
hidden_units = [1024, 1024, 1024, 1024]
# hidden_units = [2048, 2048, 2048, 2048]
# hidden_units = [20, 40, 80, 100, 100, 80, 40, 20]

if torch.backends.mps.is_available():
  device = torch.device("mps")
  x = torch.ones(1, device=device)
  print(x)
else:
  print("MPS device not found.")
  device = "cpu"

# device = "cpu"

pinn = PINN(input_dim, output_dim, hidden_units).to(device)

In [ ]:
# optimizer = torch.optim.Adam(pinn.parameters(), lr=0.001)
optimizer = torch.optim.LBFGS(pinn.parameters())

epochs = 30
Nf = Nx   # num of collocation points -> pde evaluation -> Nf^4... needs fixing: sample Nf points from the whole 3D domain
N0 = Ny   # num of points to evaluate initial conditons -> N0^4
Nb = Nx   # num of points to evaluate boundary conditions -> Nb^4
Nw = Nx   # num of points of the surface of the front wing to evaluate boundary conditions -> Nw^4

# Density (rho): 1.2041kg/m^3
# Dynamic viscosity (mu): 1.81e-5 kg/m.s
rho = 1.2
mu = 1.81e-5

# m/s
in_velocity = 80

log_filepath = "/Users/ggito/repos/pinns/data/log.csv"

In [ ]:
def sample_points_in_domain(min, max, num_of_samples):
  return np.random.uniform(min, max, size=num_of_samples)

In [ ]:
def zeros(num):
  return np.zeros(num)

In [ ]:
def ones(num):
  return np.ones(num)

In [ ]:
def create_training_inputs(x_max, y_max, z_max, t_max, Nf, N0, Nb, Nw):
  # TODO: use quasi monte carlo sampling
  # collocation points
  x_f = sample_points_in_domain(0, x_max, Nf)
  y_f = sample_points_in_domain(0, y_max, Nf)
  z_f = sample_points_in_domain(0, z_max, Nf)
  t_f = sample_points_in_domain(0, t_max, Nf)
  xyzt_f = utils.stack_xyzt(x_f, y_f, z_f, t_f)

  # initial condition points (t=0)
  x0 = sample_points_in_domain(0, x_max, N0)
  y0 = sample_points_in_domain(0, y_max, N0)
  z0 = sample_points_in_domain(0, z_max, N0)
  t0 = zeros(N0)
  xyzt_0 = utils.stack_xyzt(x0, y0, z0, t0)

  # boundary condition points (inflow, y=1)
  x_b = sample_points_in_domain(0, x_max, Nb)
  y_b = ones(Nb)
  z_b = sample_points_in_domain(0, z_max, Nb)
  t_b = sample_points_in_domain(0, t_max, Nb)
  xyzt_b = utils.stack_xyzt(x_b, y_b, z_b, t_b)

  # points & normal vectors on the surface of the wing
  ## sample Nw wing points with the corresponding normals
  sampled_indices = wing_df.sample(n=Nw).index

  x_w, y_w, z_w = [wing_df.loc[sampled_indices, col].values for col in ['x', 'y', 'z']]
  n_x, n_y, n_z = [norm_df.loc[sampled_indices, col].values for col in ['x', 'y', 'z']]
  t_w = sample_points_in_domain(0, t_max, Nw)

  xyzt_w = utils.stack_xyzt(x_w, y_w, z_w, t_w)
  n_xyz = utils.stack_xyz(n_x, n_y, n_z)

  return (utils.tensor_from_array(xyzt_f, device=device),
          utils.tensor_from_array(xyzt_0, device=device),
          utils.tensor_from_array(xyzt_b, device=device),
          utils.tensor_from_array(xyzt_w, device=device),
          utils.tensor_from_array(n_xyz, device=device))

In [ ]:
def closure(report_losses=False):
  optimizer.zero_grad()

  training_input = create_training_inputs(x_max, y_max, z_max, t_max, Nf, N0, Nb, Nw)

  total_loss, pde_loss, ic_loss, bc_loss, no_slip_loss, imp_loss = pinn.loss(
                  *training_input,
                  in_velocity,
                  mu, rho,
                  0.005, 0.25, 0.25, 0.5, 0.5,
                  log_filepath)

  total_loss.backward()

  if report_losses:
      return total_loss, pde_loss, ic_loss, bc_loss, no_slip_loss, imp_loss
  else:
      return total_loss

for epoch in range(epochs):
  optimizer.step(closure)
  # if epoch % 10 == 0:
  total_loss, pde_loss, ic_loss, bc_loss, no_slip_loss, imp_loss = closure(report_losses=True)
  print(f'Epoch: {epoch},\tTotal loss: {total_loss.item()},\tPDE loss: {pde_loss.item()},\tIC loss: {ic_loss.item()},\tBC loss: {bc_loss.item()},\tNo-slip loss: {no_slip_loss.item()},\tImpermeability loss: {imp_loss.item()}')

In [ ]:
# out_filepath = "/Users/ggito/repos/pinns/data/"
# torch.save(pinn, out_filepath + 'pinn8.pt')

In [ ]:
# out_filepath = "/Users/ggito/repos/pinns/data/"
# pinn = torch.load(out_filepath + 'pinn8.pt')
# pinn.train()
# # loaded_model.eval()